In [1]:
import time
import cv2
import torch
from tqdm import tqdm
import pytorch_warmup as warmup
import numpy as np
import random
import os

from clrnet.models.registry import build_net
from clrnet.engine.registry import build_trainer, build_evaluator
from clrnet.engine.optimizer import build_optimizer
from clrnet.engine.scheduler import build_scheduler
from clrnet.datasets import build_dataloader
from clrnet.utils.recorder import build_recorder
from clrnet.utils.net_utils import save_model, load_network, resume_network
from clrnet.utils.recorder import build_recorder
from mmcv.parallel import MMDataParallel
from nets.nn import DarkNet
from utils import util


# yolo

from utils.dataset import Dataset

/home/mmm9886/miniconda3/envs/clrnet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda:0'

In [3]:
from clrnet.utils.config import Config

cfg = Config.fromfile("configs/clrnet/clr_resnet18_culane.py")

In [4]:
cfg.load_from = None
cfg.resume_from = None
cfg.finetune_from = None
cfg.view = None
cfg.seed = 0
cfg.gpus = 1

In [5]:
torch.manual_seed(cfg.seed)
np.random.seed(cfg.seed)
random.seed(cfg.seed)

In [6]:
train_loader_lane = build_dataloader(cfg.dataset.train,
                                        cfg,
                                        is_train=True)

In [7]:
train_loader_lane

In [8]:
import argparse
parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])
args.input_size = 640
args.batch_size = 32
args.local_rank = 0
args.world_size = 1
args.epoch = 100

In [9]:
import yaml
import os
import glob
from torch.utils import data

with open(os.path.join('utils', 'args.yaml'), errors='ignore') as f:
    params = yaml.safe_load(f)

folder_path = "/data/bdd100k/images/train2017/"
filenames = glob.glob(os.path.join(folder_path, '*'))

train_dataset_obj = Dataset(filenames, args.input_size, params, True)

In [10]:
train_loader_obj = data.DataLoader(train_dataset_obj, 18, False, num_workers=8,
                             pin_memory=True, collate_fn=Dataset.collate_fn)

In [11]:
train_loader_obj

In [23]:
state_dict = torch.load("/home/mmm9886/Desktop/Capstone_Implementation/YOLOv8-pt copy/weights/best.pt")
darknet_backbone = state_dict['model'].net
darknet_backbone = darknet_backbone.float()
darknet_backbone = darknet_backbone.to(device)
darknet_neck = state_dict['model'].fpn
darknet_neck = darknet_neck.float()
darknet_neck = darknet_neck.to(device)
darknet_head = state_dict['model'].head
darknet_head = darknet_head.float()
darknet_head = darknet_head.to(device)

In [24]:
for params in state_dict['model'].parameters():
    print(params.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [25]:
state_dict = torch.load("/home/mmm9886/Desktop/Capstone_Implementation/YOLOv8-pt copy 3/weights/best.pt")

In [26]:
net = build_net(cfg)

pretrained model:  https://download.pytorch.org/models/resnet18-5c106cde.pth


In [27]:
net.parameters()

<generator object Module.parameters at 0x7f28d40cae40>

In [28]:
print(sum(p.numel() for p in net.parameters()))
print(sum(p.numel() for p in state_dict["model"].parameters()))


11774387
3011043


In [30]:
print(param_size_clrnet, param_size_yolov8n)

47097548 6022086


In [19]:
buffer_size

130108

In [29]:
net.heads

CLRHead(
  (prior_embeddings): Embedding(192, 3)
  (seg_decoder): SegDecoder(
    (dropout): Dropout2d(p=0.1, inplace=False)
    (conv): Conv2d(192, 5, kernel_size=(1, 1), stride=(1, 1))
  )
  (reg_modules): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU(inplace=True)
  )
  (cls_modules): ModuleList(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU(inplace=True)
  )
  (roi_gather): ROIGather(
    (f_key): ConvModule(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activate): ReLU(inplace=True)
    )
    (f_query): Sequential(
      (0): Conv1d(192, 192, kernel_size=(1,), stride=(1,), groups=192)
      (1): ReLU()
    )
    (f

In [30]:
# from collections import OrderedDict
# state_dict = torch.load('/home/mmm9886/Desktop/Capstone_Implementation/clrnet/work_dirs/clr/r18_culane/20250111_141413_lr_6e-04_b_24/ckpt/9.pth', map_location=torch.device("cuda:1"))
# prefix_to_keep = "module.heads."
# filtered_od = OrderedDict((k, v) for k, v in state_dict["net"].items() if k.startswith(prefix_to_keep))
# filtered_od = {k.replace("module.heads.", ""): v for k, v in filtered_od.items()}


In [31]:
clrnet_head = net.heads
# clrnet_head.load_state_dict(filtered_od)

In [32]:
class YOLOL(torch.nn.Module):
    def __init__(self, backbone, neck, yolo_head, clrhead):
        super(YOLOL, self).__init__()
        self.backbone = backbone
        self.neck = neck
        self.yolo_head = yolo_head
        self.clrhead = clrhead
        
        self.offset_2 = torch.nn.Conv2d(512, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        self.offset_1 = torch.nn.Conv2d(256, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) 
        self.offset_0 = torch.nn.Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    def forward(self, x, task_id):
        # Get the outputs from the neck
        out = self.backbone(x['img'] if isinstance(x, dict) else x)
        out0, out1, out2 = self.neck(out)
        
        
        # Apply Conv2d layers to each output
        if task_id == 0: # lane detection
            out0 = self.offset_0(out0)
            out1 = self.offset_1(out1)
            out2 = self.offset_2(out2)
            if self.training:
                output = self.clrhead([out0, out1, out2], batch = x)
            else:
                output = self.clrhead([out0, out1, out2])
        else: # object detection
            output = self.yolo_head([out0, out1, out2])
        
        return output
    
def reset_weights(m):
    if isinstance(m, torch.nn.Linear):  # Add other layer types if needed
        torch.nn.init.kaiming_normal_(m.weight)  # He initialization
        if m.bias is not None:
            torch.nn.init.zeros_(m.bias)  # Reset bias to 0
        
       

In [35]:
from multitask import YOLOL
model_pc = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)
model_pc.load_state_dict(torch.load("/home/mmm9886/Desktop/Capstone_Implementation/clrnet/weights/pcgrad_control_v8n_model2_best.pt"))

<All keys matched successfully>

In [36]:
from multitask_cross_stitch import YOLOL
model_pc_cs = YOLOL(darknet_backbone, darknet_head, clrnet_head)
model_pc_cs_scale = YOLOL(darknet_backbone, darknet_head, clrnet_head)

model_pc_cs.load_state_dict(torch.load("/home/mmm9886/Desktop/Capstone_Implementation/clrnet/weights/pcgrad_divergentneck_cross_stitch_model2_v8n_best.pt"))
model_pc_cs_scale.load_state_dict(torch.load("/home/mmm9886/Desktop/Capstone_Implementation/clrnet/weights/pcgrad_divergentneck_cross_stitch_model_scaled_v8n_best.pt"))

<All keys matched successfully>

In [18]:
from multitask_cross_stitch import YOLOL
model_cross_stitch = YOLOL(darknet_backbone, darknet_head, clrnet_head)
model_cross_stitch.load_state_dict(torch.load('/home/mmm9886/Desktop/Capstone_Implementation/clrnet/weights/pcgrad_divergentneck_cross_stitch_model_scaled2_v8n_last.pt'))

# from multitask import YOLOL
# model_control = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)
# model_control.load_state_dict(torch.load('/home/mmm9886/Desktop/Capstone_Implementation/clrnet/weights/pcgrad_control_v8n_model2_best.pt'))

<All keys matched successfully>

In [ ]:
from multitask_s import YOLOL
clrnet_yolo = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)



RuntimeError: Error(s) in loading state_dict for YOLOL:
	Missing key(s) in state_dict: "backbone.p1.0.conv.weight", "backbone.p1.0.norm.weight", "backbone.p1.0.norm.bias", "backbone.p1.0.norm.running_mean", "backbone.p1.0.norm.running_var", "backbone.p2.0.conv.weight", "backbone.p2.0.norm.weight", "backbone.p2.0.norm.bias", "backbone.p2.0.norm.running_mean", "backbone.p2.0.norm.running_var", "backbone.p2.1.conv1.conv.weight", "backbone.p2.1.conv1.norm.weight", "backbone.p2.1.conv1.norm.bias", "backbone.p2.1.conv1.norm.running_mean", "backbone.p2.1.conv1.norm.running_var", "backbone.p2.1.conv2.conv.weight", "backbone.p2.1.conv2.norm.weight", "backbone.p2.1.conv2.norm.bias", "backbone.p2.1.conv2.norm.running_mean", "backbone.p2.1.conv2.norm.running_var", "backbone.p2.1.conv3.conv.weight", "backbone.p2.1.conv3.norm.weight", "backbone.p2.1.conv3.norm.bias", "backbone.p2.1.conv3.norm.running_mean", "backbone.p2.1.conv3.norm.running_var", "backbone.p2.1.res_m.0.res_m.0.conv.weight", "backbon

In [21]:
state_dict = torch.load('/home/mmm9886/Desktop/Capstone_Implementation/clrnet/work_dirs/clr/r18_culane/20250111_141413_lr_6e-04_b_24_________/ckpt/9.pth')['net']
from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in state_dict.items():
    new_key = k.replace("module.", "")  # remove 'module.'
    new_state_dict[new_key] = v
    
clrnet_yolo.load_state_dict(new_state_dict)

RuntimeError: Error(s) in loading state_dict for YOLOL:
	Missing key(s) in state_dict: "neck.h1.conv1.conv.weight", "neck.h1.conv1.norm.weight", "neck.h1.conv1.norm.bias", "neck.h1.conv1.norm.running_mean", "neck.h1.conv1.norm.running_var", "neck.h1.conv2.conv.weight", "neck.h1.conv2.norm.weight", "neck.h1.conv2.norm.bias", "neck.h1.conv2.norm.running_mean", "neck.h1.conv2.norm.running_var", "neck.h1.conv3.conv.weight", "neck.h1.conv3.norm.weight", "neck.h1.conv3.norm.bias", "neck.h1.conv3.norm.running_mean", "neck.h1.conv3.norm.running_var", "neck.h1.res_m.0.res_m.0.conv.weight", "neck.h1.res_m.0.res_m.0.norm.weight", "neck.h1.res_m.0.res_m.0.norm.bias", "neck.h1.res_m.0.res_m.0.norm.running_mean", "neck.h1.res_m.0.res_m.0.norm.running_var", "neck.h1.res_m.0.res_m.1.conv.weight", "neck.h1.res_m.0.res_m.1.norm.weight", "neck.h1.res_m.0.res_m.1.norm.bias", "neck.h1.res_m.0.res_m.1.norm.running_mean", "neck.h1.res_m.0.res_m.1.norm.running_var", "neck.h2.conv1.conv.weight", "neck.h2.conv1

In [19]:
clrnet_yolo

YOLOL(
  (backbone): DarkNet(
    (p1): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (relu): SiLU(inplace=True)
      )
    )
    (p2): Sequential(
      (0): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (relu): SiLU(inplace=True)
      )
      (1): CSP(
        (conv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (relu): SiLU(inplace=True)
        )
        (conv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm): BatchNorm2d(32, eps=0.001, moment

In [ ]:
model_cross_stitch

YOLOL(
  (backbone): DarkNet(
    (p1): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (relu): SiLU(inplace=True)
      )
    )
    (p2): Sequential(
      (0): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (norm): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (relu): SiLU(inplace=True)
      )
      (1): CSP(
        (conv1): Conv(
          (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (relu): SiLU(inplace=True)
        )
        (conv2): Conv(
          (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm): BatchNorm2d(16, eps=0.001, moment

: 

In [ ]:
p = [], [], []
for v in net.modules():
    if hasattr(v, 'bias') and isinstance(v.bias, torch.nn.Parameter):
        p[2].append(v.bias)
    if isinstance(v, torch.nn.BatchNorm2d):
        p[1].append(v.weight)
    elif hasattr(v, 'weight') and isinstance(v.weight, torch.nn.Parameter):
        p[0].append(v.weight)

In [ ]:
net = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)

In [ ]:
state_dict = torch.load('weights/vehicle_pcgrad_model_best.pt')

In [ ]:
model = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)  # Replace with your model class
model.load_state_dict(torch.load('weights/vehicle_only_model_best.pt'))
# model.eval()  # Set model to evaluation mode

RuntimeError: Error(s) in loading state_dict for YOLOL:
	size mismatch for backbone.p1.0.conv.weight: copying a param with shape torch.Size([32, 3, 3, 3]) from checkpoint, the shape in current model is torch.Size([16, 3, 3, 3]).
	size mismatch for backbone.p1.0.norm.weight: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for backbone.p1.0.norm.bias: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for backbone.p1.0.norm.running_mean: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for backbone.p1.0.norm.running_var: copying a param with shape torch.Size([32]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for backbone.p2.0.conv.weight: copying a param with shape torch.Size([64, 32, 3, 3]) from checkpoint, the shape in current mode

In [ ]:
from torchvision import transforms


# Example transform (adapt if your CLRNet expects specific normalization)
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((288, 800)),  # Resize like training
    transforms.ToTensor(),          # Convert to [C, H, W] and scale [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Imagenet means
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
img_path = "/data/bdd100k/images/val2017/b3d67c06-57b140bf.jpg"  # Change path

# Read using OpenCV (BGR)
image_bgr = cv2.imread(img_path)

# Convert BGR → RGB
image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

# Keep a copy for visualization later
image_vis = image.copy()

# Preprocess
input_tensor = preprocess(image)
input_tensor = input_tensor.unsqueeze(0).cuda()  # Add batch dim and send to GPU


In [ ]:
print(input_tensor.shape)
print(next(model.parameters()).device)
print(input_tensor.device)


In [ ]:
model = model.cuda()
model.eval()  # Set model to evaluation mode
# with torch.no_grad():
output_lane = model(input_tensor, 0)
    # output_obj = model(input_tensor, 1)

In [ ]:
import torch.nn as nn
softmax = nn.Softmax(dim=1)
for pred in output_lane:
    print(pred.shape)
    print(softmax(pred[:, :2])[:, 1].shape)

In [ ]:
model.clrhead.get_lanes(output_lane)

In [ ]:
output_lane["seg"].shape

In [ ]:
len(output_lane["predictions_lists"])

In [ ]:
output_lane["predictions_lists"][0].shape

In [ ]:
output_lane.shape

In [ ]:
output_obj.shape

In [37]:
recorder = build_recorder(cfg)

2025-05-11 18:22:40,009 - clrnet.utils.recorder - INFO - Config: 
net = dict(type='Detector', )

backbone = dict(
    type='ResNetWrapper',
    resnet='resnet18',
    pretrained=True,
    replace_stride_with_dilation=[False, False, False],
    out_conv=False,
)

num_points = 72
max_lanes = 4
sample_y = range(589, 230, -20)

heads = dict(type='CLRHead',
             num_priors=192,
             refine_layers=3,
             fc_hidden_dim=64,
             sample_points=36)

iou_loss_weight = 2.
cls_loss_weight = 2.
xyt_loss_weight = 0.2
seg_loss_weight = 1.0

work_dirs = "work_dirs/clr/r18_culane"

neck = dict(type='FPN',
            in_channels=[128, 256, 512],
            out_channels=64,
            num_outs=3,
            attention=False)

test_parameters = dict(conf_threshold=0.4, nms_thres=50, nms_topk=max_lanes)

epochs = 15
batch_size = 24

optimizer = dict(type='AdamW', lr=0.6e-3)  # 3e-4 for batchsize 8
total_iter = (88880 // batch_size) * epochs
scheduler = dict(type='CosineAn

In [ ]:
print(len(train_loader_obj))
print(len(train_loader_lane))

3889
3704


In [38]:
def to_cuda(batch, device = 'cpu'):
    for k in batch:
        if not isinstance(batch[k], torch.Tensor):
            continue
        batch[k] = batch[k].to(device)
    return batch

In [39]:
# p_bar = enumerate(train_dataset_obj)
# p_bar = tqdm(p_bar, total=len(train_loader_obj))  # progress bar
# for i, (samples, targets, _) in p_bar:
#     print(samples)
#     print(targets)
#     break

In [40]:
# for i, data in enumerate(train_loader_lane):
#     print(data)
#     data = to_cuda(data)
#     print(data)
#     break

In [ ]:
for param in net.backbone.parameters():
    param.requires_grad = True
for param in net.neck.parameters():
    param.requires_grad = True
for param in net.yolo_head.parameters():
    param.requires_grad = True
    

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
image_path = '/data/bdd100k/images/val2017/c8054f9c-3ff3c2b1.jpg'

# Load the image with PIL
image = Image.open(image_path).convert('RGB')

transform = transforms.Compose([
    transforms.Resize((224, 224)),   # adjust size as required by model
    transforms.ToTensor(),           # converts PIL image to a PyTorch tensor, CxHxW
    transforms.Normalize(            # normalize with mean & std used by model training
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

input_tensor = transform(image)
input_batch = input_tensor.unsqueeze(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_batch = input_batch.to(device)
input_batch = input_batch.half()

In [ ]:
model_cross_stitch.half()
model_cross_stitch.cuda()
model_cross_stitch.eval()
output = model_cross_stitch(input_batch, 1)

In [ ]:
from torchviz import make_dot


In [ ]:
make_dot(output, params=dict(model_cross_stitch.named_parameters())).render("model_arch", format="png")


'model_arch.png'

In [ ]:
class WrappedModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        # call original model with default or dummy extra args
        return self.model(x, 1)

In [ ]:
model = WrappedModel(model_cross_stitch)
model = model.cuda()
model = model.half()
model.eval()

WrappedModel(
  (model): YOLOL(
    (backbone): DarkNet(
      (p1): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (norm): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (relu): SiLU(inplace=True)
        )
      )
      (p2): Sequential(
        (0): Conv(
          (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (norm): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (relu): SiLU(inplace=True)
        )
        (1): CSP(
          (conv1): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (relu): SiLU(inplace=True)
          )
          (conv2): Conv(
            (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1

In [ ]:
import hiddenlayer as hl
hl.build_graph(model, torch.randn(1, 3, 224, 224).cuda().half()).save("arch", format="png")

/home/mmm9886/miniconda3/envs/clrnet/lib/python3.8/site-packages/torch/tensor.py:587: RuntimeWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  warnings.warn('Iterating over a tensor might cause the trace to be incorrect. '
/home/mmm9886/miniconda3/envs/clrnet/lib/python3.8/site-packages/torch/onnx/symbolic_helper.py:677: UserWarning: ONNX export mode is set to inference mode, but operator batch_norm is set to inference mode. The model will be exported in inference, as specified by the export mode.
  warnings.warn("ONNX export mode is set to " + training_mode +
/home/mmm9886/miniconda3/envs/clrnet/lib/python3.8/site-packages/torch/onnx/symbolic_helper.py:346: UserWarning: You are trying to export the model with onnx:Upsample for ONNX opset version 9. This operator might cause results to not match the expected results by

In [ ]:
net.apply(reset_weights)

In [ ]:
import numpy

@torch.no_grad()
def test_obj(args, params, model=None):
    
    folder_path = "/data/bdd100k/images/val2017/"
    filenames = glob.glob(os.path.join(folder_path, '*'))

    val_dataset_obj = Dataset(filenames, args.input_size, params, False)

    loader = data.DataLoader(val_dataset_obj, 8, False, num_workers=8,
                             pin_memory=True, collate_fn=Dataset.collate_fn)

    if model is None:
        model = torch.load('./weights/best.pt', map_location='cuda')['model'].float()

    model.half()
    model.eval()

    # Configure
    iou_v = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
    n_iou = iou_v.numel()

    m_pre = 0.
    m_rec = 0.
    map50 = 0.
    mean_ap = 0.
    metrics = []
    p_bar = tqdm(loader, desc=('%10s' * 3) % ('precision', 'recall', 'mAP'))
    for samples, targets, shapes in p_bar:
        samples = samples.to(device)
        targets = targets.to(device)
        samples = samples.half()  # uint8 to fp16/32
        samples = samples / 255  # 0 - 255 to 0.0 - 1.0
        _, _, height, width = samples.shape  # batch size, channels, height, width

        # Inference
        outputs = model(samples, 1)
        # NMS
        targets[:, 2:] *= torch.tensor((width, height, width, height)).to(device) # to pixels
        outputs = util.non_max_suppression(outputs, 0.25, 0.45) # 0.001 originally
        # Metrics
        for i, output in enumerate(outputs):
            labels = targets[targets[:, 0] == i, 1:]
            correct = torch.zeros(output.shape[0], n_iou, dtype=torch.bool).to(device)

            if output.shape[0] == 0:
                if labels.shape[0]:
                    metrics.append((correct, *torch.zeros((3, 0)).to(device)))
                continue

            detections = output.clone()
            util.scale(detections[:, :4], samples[i].shape[1:], shapes[i][0], shapes[i][1])

            # Evaluate
            if labels.shape[0]:
                tbox = labels[:, 1:5].clone()  # target boxes
                tbox[:, 0] = labels[:, 1] - labels[:, 3] / 2  # top left x
                tbox[:, 1] = labels[:, 2] - labels[:, 4] / 2  # top left y
                tbox[:, 2] = labels[:, 1] + labels[:, 3] / 2  # bottom right x
                tbox[:, 3] = labels[:, 2] + labels[:, 4] / 2  # bottom right y
                util.scale(tbox, samples[i].shape[1:], shapes[i][0], shapes[i][1])

                correct = numpy.zeros((detections.shape[0], iou_v.shape[0]))
                correct = correct.astype(bool)

                t_tensor = torch.cat((labels[:, 0:1], tbox), 1)
                iou = util.box_iou(t_tensor[:, 1:], detections[:, :4])
                correct_class = t_tensor[:, 0:1] == detections[:, 5]
                for j in range(len(iou_v)):
                    x = torch.where((iou >= iou_v[j]) & correct_class)
                    if x[0].shape[0]:
                        matches = torch.cat((torch.stack(x, 1), iou[x[0], x[1]][:, None]), 1)
                        matches = matches.cpu().numpy()
                        if x[0].shape[0] > 1:
                            matches = matches[matches[:, 2].argsort()[::-1]]
                            matches = matches[numpy.unique(matches[:, 1], return_index=True)[1]]
                            matches = matches[numpy.unique(matches[:, 0], return_index=True)[1]]
                        correct[matches[:, 1].astype(int), j] = True
                correct = torch.tensor(correct, dtype=torch.bool, device=iou_v.device)
            metrics.append((correct, output[:, 4], output[:, 5], labels[:, 0]))

    # Compute metrics
    metrics = [torch.cat(x, 0).cpu().numpy() for x in zip(*metrics)]  # to numpy
    if len(metrics) and metrics[0].any():
        tp, fp, m_pre, m_rec, map50, mean_ap = util.compute_ap(*metrics)

    # Print results
    print('%10.3g' * 3 % (m_pre, m_rec, mean_ap))

    # Return results
    model.float()  # for training
    return tp, fp, m_pre, m_rec, map50, mean_ap

In [41]:
@torch.no_grad()
def validate_lane(net, recorder, device):
    
    net.backbone = net.backbone.to(device)
    net.neck = net.neck.to(device)
    net.clrhead= net.clrhead.to(device)
    net.yolo_head = net.yolo_head.to(device)
    # net.offset_0 = net.offset_0.to(device)
    net.offset_1 = net.offset_1.to(device)
    net.offset_2 = net.offset_2.to(device)
    
    val_loader_lane = build_dataloader(cfg.dataset.val,
                                            cfg,
                                            is_train=False)
    predictions = []
    for i, data in enumerate(tqdm(val_loader_lane, desc=f'Validate')):
        data = to_cuda(data, device)
        with torch.no_grad():
            output = net(data, 0)
            output = net.clrhead.get_lanes(output)
            predictions.extend(output)
        if cfg.view:
            val_loader_lane.dataset.view(output, data['meta'])

    metric = val_loader_lane.dataset.evaluate(predictions,
                                                cfg.work_dir)
    # recorder.logger.info('metric: ' + str(metric))
    return metric

In [ ]:
val_loader_lane = build_dataloader(cfg.dataset.val,
                                            cfg,
                                            is_train=False)

2025-05-04 00:37:29,067 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...


In [ ]:
val_loader_lane.dataset

In [42]:
model_pc.eval()
metric = validate_lane(model_pc, recorder, device)

2025-05-11 18:24:00,480 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [02:12<00:00, 10.93it/s]


Generating prediction output...


2025-05-11 18:26:16,774 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-11 18:26:54,130 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 27, fp: 3526, fn: 32750,precision: 0.007599211933577259, recall: 0.0008237483601305793, f1: 0.001486374896779521
2025-05-11 18:26:54,200 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-11 18:27:27,289 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 2, fp: 1009, fn: 28001,precision: 0.0019782393669634025, recall: 7.142091918722994e-05, f1: 0.00013786447921692975
2025-05-11 18:27:27,348 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-11 18:27:55,235 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 1, fp: 145, fn: 1684,precision: 0.00684931506849315, recall: 0.0005934718100890207, f1: 0.0010922992900054614
2025-0

In [43]:
model_pc_cs.eval()
metric_pc_cs = validate_lane(model_pc_cs, recorder, device)

2025-05-11 18:30:49,404 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [03:16<00:00,  7.36it/s]


Generating prediction output...


2025-05-11 18:34:15,738 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-11 18:35:16,818 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 28958, fp: 3203, fn: 3819,precision: 0.9004073256428594, recall: 0.883485370839308, f1: 0.8918660876528381
2025-05-11 18:35:16,955 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-11 18:36:09,523 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 18254, fp: 4531, fn: 9749,precision: 0.8011411016019311, recall: 0.6518587294218476, f1: 0.718831219973222
2025-05-11 18:36:09,632 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-11 18:36:41,127 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 936, fp: 293, fn: 749,precision: 0.7615947925142392, recall: 0.5554896142433234, f1: 0.6424159231297186
2025-05-11 18:36:41,136 - 

In [44]:
model_pc_cs_scale.eval()
metric_pc_cs_scale = validate_lane(model_pc_cs_scale, recorder, device)

2025-05-11 18:40:07,990 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [03:15<00:00,  7.40it/s]


Generating prediction output...


2025-05-11 18:43:32,968 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-11 18:44:34,617 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 28679, fp: 2742, fn: 4098,precision: 0.9127335221667038, recall: 0.8749733044512921, f1: 0.8934546247546653
2025-05-11 18:44:34,751 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-11 18:45:27,708 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 17655, fp: 3937, fn: 10348,precision: 0.817663949610967, recall: 0.6304681641252723, f1: 0.7119669321504184
2025-05-11 18:45:27,812 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-11 18:46:00,395 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 855, fp: 205, fn: 830,precision: 0.8066037735849056, recall: 0.5074183976261127, f1: 0.6229508196721312
2025-05-11 18:46:00,404 

In [33]:
model_cross_stitch.eval()
metric = validate_lane(model_cross_stitch, recorder, device)

2025-05-04 01:33:45,894 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [03:03<00:00,  7.86it/s]


Generating prediction output...


2025-05-04 01:37:02,159 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-04 01:38:00,812 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 1468, fp: 33214, fn: 31309,precision: 0.042327432097341564, recall: 0.04478750343228483, f1: 0.043522732326301904
2025-05-04 01:38:00,952 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-04 01:38:54,687 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 1090, fp: 27733, fn: 26913,precision: 0.03781702112895951, recall: 0.03892440095704032, f1: 0.03836272128955056
2025-05-04 01:38:54,813 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-04 01:39:21,732 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 71, fp: 1511, fn: 1614,precision: 0.04487989886219975, recall: 0.042136498516320474, f1: 0.04346495255586165
2025-05-0

In [31]:
metric

{'normal': {0.5: {'TP': 1468,
   'FP': 33214,
   'FN': 31309,
   'Precision': 0.042327432097341564,
   'Recall': 0.04478750343228483,
   'F1': 0.043522732326301904}},
 'crowd': {0.5: {'TP': 1090,
   'FP': 27733,
   'FN': 26913,
   'Precision': 0.03781702112895951,
   'Recall': 0.03892440095704032,
   'F1': 0.03836272128955056}},
 'hlight': {0.5: {'TP': 71,
   'FP': 1511,
   'FN': 1614,
   'Precision': 0.04487989886219975,
   'Recall': 0.042136498516320474,
   'F1': 0.04346495255586165}},
 'shadow': {0.5: {'TP': 93,
   'FP': 2946,
   'FN': 2783,
   'Precision': 0.03060217176702863,
   'Recall': 0.032336578581363004,
   'F1': 0.03144547759932376}},
 'noline': {0.5: {'TP': 611,
   'FP': 12661,
   'FN': 13410,
   'Precision': 0.046036769138034964,
   'Recall': 0.043577490906497394,
   'F1': 0.04477338511706299}},
 'arrow': {0.5: {'TP': 153,
   'FP': 2979,
   'FN': 3029,
   'Precision': 0.04885057471264368,
   'Recall': 0.04808296668761785,
   'F1': 0.04846373139056066}},
 'curve': {0.5: {'

In [ ]:
model_cross_stitch.eval()
metric = validate_lane(model_cross_stitch, recorder, device)

2025-05-04 00:09:58,243 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...


Validate: 100%|██████████| 1445/1445 [02:48<00:00,  8.59it/s]


Generating prediction output...


2025-05-04 00:12:56,785 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-04 00:13:47,595 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 28679, fp: 2742, fn: 4098,precision: 0.9127335221667038, recall: 0.8749733044512921, f1: 0.8934546247546653
2025-05-04 00:13:47,739 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-04 00:14:29,380 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 17655, fp: 3937, fn: 10348,precision: 0.817663949610967, recall: 0.6304681641252723, f1: 0.7119669321504184
2025-05-04 00:14:29,488 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-04 00:14:49,610 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 855, fp: 205, fn: 830,precision: 0.8066037735849056, recall: 0.5074183976261127, f1: 0.6229508196721312
2025-05-04 00:14:49,620 

In [ ]:
metric

0.7371510914233868

In [ ]:
model_cross_stitch.eval()
metric2 = validate_lane(model_cross_stitch, recorder, device)

2025-05-04 00:19:46,117 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [02:12<00:00, 10.90it/s]


Generating prediction output...


2025-05-04 00:22:04,095 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-04 00:22:35,029 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 140, fp: 7218, fn: 32637,precision: 0.019026909486273443, recall: 0.004271287793269671, f1: 0.006976454466176654
2025-05-04 00:22:35,112 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-04 00:23:02,167 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 21, fp: 4059, fn: 27982,precision: 0.005147058823529412, recall: 0.0007499196514659144, f1: 0.0013091045101767292
2025-05-04 00:23:02,232 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-04 00:23:21,591 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 2, fp: 231, fn: 1683,precision: 0.008583690987124463, recall: 0.0011869436201780415, f1: 0.0020855057351407713
2025-0

In [ ]:
model_cross_stitch.eval()
validate_lane(model_cross_stitch, recorder, device)

2025-05-03 15:57:40,013 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [02:46<00:00,  8.67it/s]


Generating prediction output...


2025-05-03 16:00:37,162 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-03 16:01:30,042 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 28486, fp: 2522, fn: 4291,precision: 0.9186661506707946, recall: 0.8690850291362846, f1: 0.8931880536176218
2025-05-03 16:01:30,172 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-03 16:02:14,223 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 17385, fp: 3847, fn: 10618,precision: 0.8188112283345893, recall: 0.6208263400349963, f1: 0.7062049355133544
2025-05-03 16:02:14,329 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-03 16:02:37,566 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 888, fp: 238, fn: 797,precision: 0.7886323268206039, recall: 0.5270029673590505, f1: 0.6318036286019212
2025-05-03 16:02:37,576

0.7329673219451198

In [ ]:
model_cross_stitch.eval()
validate_lane(model_cross_stitch, recorder, device)

2025-05-03 14:36:20,647 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...


Validate: 100%|██████████| 1445/1445 [02:48<00:00,  8.57it/s]


Generating prediction output...


2025-05-03 14:39:19,506 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-03 14:40:12,580 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 28905, fp: 2736, fn: 3872,precision: 0.9135299137195411, recall: 0.8818683833175702, f1: 0.8974199757831661
2025-05-03 14:40:12,719 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-03 14:40:56,588 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 18007, fp: 4200, fn: 9996,precision: 0.8108704462556852, recall: 0.6430382459022248, f1: 0.7172674765982873
2025-05-03 14:40:56,694 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-03 14:41:18,952 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 912, fp: 240, fn: 773,precision: 0.7916666666666666, recall: 0.5412462908011869, f1: 0.6429326753612972
2025-05-03 14:41:18,963 

0.7402799706405742

In [ ]:
model_cross_stitch.eval()
validate_lane(model_cross_stitch, recorder, device)

2025-05-03 15:11:01,567 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [02:47<00:00,  8.63it/s]


Generating prediction output...


2025-05-03 15:13:58,916 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-03 15:14:52,055 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 28920, fp: 2706, fn: 3857,precision: 0.9144374881426675, recall: 0.8823260212954206, f1: 0.8980948092480163
2025-05-03 15:14:52,186 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-03 15:15:37,395 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 17856, fp: 4178, fn: 10147,precision: 0.8103839520740673, recall: 0.6376459665035888, f1: 0.7137118532286107
2025-05-03 15:15:37,499 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-03 15:16:00,798 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 924, fp: 245, fn: 761,precision: 0.7904191616766467, recall: 0.5483679525222552, f1: 0.6475122634898389
2025-05-03 15:16:00,812

0.7409400372124493

In [ ]:
model_control.eval()
validate_lane(model_control, recorder, device)

2025-05-01 12:25:18,866 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [02:28<00:00,  9.70it/s]


Generating prediction output...


2025-05-01 12:27:56,313 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-01 12:28:56,525 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 28977, fp: 1771, fn: 3800,precision: 0.942402757902953, recall: 0.8840650456112518, f1: 0.9123022432113341
2025-05-01 12:28:56,637 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-01 12:29:46,755 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 17293, fp: 2207, fn: 10710,precision: 0.8868205128205128, recall: 0.6175409777523837, f1: 0.7280803317685199
2025-05-01 12:29:46,843 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-01 12:30:18,317 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 894, fp: 211, fn: 791,precision: 0.8090497737556561, recall: 0.5305637982195845, f1: 0.6408602150537633
2025-05-01 12:30:18,325 

0.7506859642023691

In [ ]:
model_cross_stitch.eval()
validate_lane(model_cross_stitch, recorder, device)

2025-05-01 12:38:56,826 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [02:41<00:00,  8.97it/s]


Generating prediction output...


2025-05-01 12:41:47,110 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-01 12:42:47,622 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 29739, fp: 2439, fn: 3038,precision: 0.9242028715271303, recall: 0.9073130548860482, f1: 0.9156800862135325
2025-05-01 12:42:47,738 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-01 12:43:41,651 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 19346, fp: 5564, fn: 8657,precision: 0.776635889201124, recall: 0.6908545512980752, f1: 0.7312380700394988
2025-05-01 12:43:41,743 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-01 12:44:12,062 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 1055, fp: 316, fn: 630,precision: 0.7695113056163384, recall: 0.6261127596439169, f1: 0.6904450261780105
2025-05-01 12:44:12,070 

In [ ]:
model_cross_stitch.eval()
validate_lane(model_cross_stitch, recorder, device)

2025-05-01 20:02:46,553 - clrnet.datasets.base_dataset - INFO - Loading CULane annotations...
Validate: 100%|██████████| 1445/1445 [02:42<00:00,  8.89it/s]


Generating prediction output...


2025-05-01 20:05:38,885 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test0_normal.txt
2025-05-01 20:06:28,943 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 29283, fp: 2511, fn: 3494,precision: 0.9210228344970749, recall: 0.8934008603593984, f1: 0.9070015951433305
2025-05-01 20:06:29,073 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test1_crowd.txt
2025-05-01 20:07:12,859 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 18857, fp: 4463, fn: 9146,precision: 0.8086192109777015, recall: 0.6733921365567975, f1: 0.7348362332677357
2025-05-01 20:07:12,959 - clrnet.utils.culane_metric - INFO - Calculating metric for List: ./data/CULane/list/test_split/test2_hlight.txt
2025-05-01 20:07:33,494 - clrnet.utils.culane_metric - INFO - iou thr: 0.50, tp: 998, fp: 285, fn: 687,precision: 0.7778643803585347, recall: 0.5922848664688427, f1: 0.6725067385444744
2025-05-01 20:07:33,503 

0.7537217098003289

In [ ]:
sum(p.numel() for p in model_cross_stitch.parameters())

4427957

In [ ]:
sum(p.numel() for p in model_control.parameters())

3663537

In [ ]:
# vehicle_pcgrad 
model.eval()
validate_lane(model, recorder, device)

In [ ]:
# vehicle only
model.eval()
validate_lane(model, recorder, device)

In [ ]:
model.eval()
# conf_threshold=0.1, iou_threshold=0.65
test_obj(args, params, model)

In [ ]:
# conf_threshold=0.25, iou_threshold=0.65
test_obj(args, params, model)

In [ ]:
# conf_threshold=0.001, iou_threshold=0.65
test_obj(args, params, model)

In [ ]:
# conf_threshold=0.25, iou_threshold=0.45
test_obj(args, params, model)

In [ ]:
# conf_threshold=0.25, iou_threshold=0.5
test_obj(args, params, model)

In [ ]:
# conf_threshold=0.3, iou_threshold=0.5
test_obj(args, params, model)

vehicle only

In [ ]:
# conf_threshold=0.3, iou_threshold=0.5
test_obj(args, params, model)

In [ ]:
# conf_threshold=0.3, iou_threshold=0.45
test_obj(args, params, model)

In [ ]:
# conf_threshold=0.25, iou_threshold=0.45
test_obj(args, params, model)

In [ ]:
# device = 'cpu'
# net.backbone = net.backbone.to(device)
# net.neck = net.neck.to(device)
# net.clrhead= net.clrhead.to(device)
# net.yolo_head = net.yolo_head.to(device)
# net.offset_0 = net.offset_0.to(device)
# net.offset_1 = net.offset_1.to(device)
# net.offset_2 = net.offset_2.to(device)

# devices = {param.device for param in net.clrhead.parameters()}
# print(devices)

In [ ]:
net.eval()
validate_lane(net, recorder, 'cpu')

In [ ]:
accumulate = max(round(64 / (args.batch_size * args.world_size)), 1)
net = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)
net = net.to(device)

optimizer = build_optimizer(cfg, net)
scheduler = build_scheduler(cfg, optimizer)
criterion_obj = util.ComputeLoss(net, params)

net.eval()

# test_obj(args, params, net)
validate_lane()

In [ ]:

# net = MMDataParallel(net,
#                                 #   device_ids=[2]).cuda(torch.device('cuda:2'))
#                                   device_ids=range(cfg.gpus)).cuda()


accumulate = max(round(64 / (args.batch_size * args.world_size)), 1)
net = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)
net = net.to(device)

optimizer = build_optimizer(cfg, net)
scheduler = build_scheduler(cfg, optimizer)
criterion_obj = util.ComputeLoss(net, params)


end = time.time()
max_iter = len(train_loader_lane)

f = open("all_loss.txt", "a")
print("total_loss, loss_cls_obj, loss_box_obj, loss_dfl_obj, loss_cls_lane, loss_reg_xytl_lane, loss_seg_lane, loss_iou_lane")
f.write("total_loss, loss_cls_obj, loss_box_obj, loss_dfl_obj, loss_cls_lane, loss_reg_xytl_lane, loss_seg_lane, loss_iou_lane")

for epoch in range(args.epoch):
    if epoch == 1:
        break
    
    
    net.train()
    zipped_dls = zip(train_loader_obj, train_loader_lane)
    
    loss_scaler = 0.2

    p_bar = enumerate(zipped_dls)
    p_bar = tqdm(p_bar, total=len(train_loader_lane))
    

    # for i, data in enumerate(train_loader_lane):
    for i, (data_obj, data_lane) in p_bar:
        x = i + max_iter #* epoch
        date_time = time.time() - end
        samples = data_obj[0].to(device).float() / 255
        targets = data_obj[1].to(device)
        # data_obj = to_cuda(data_obj)
        data_lane = to_cuda(data_lane, device)
        
        
        
        # experiment = net2(data_lane)
        output_lane = net(data_lane, 0)
        output_obj = net(samples, 1)
        
        amp_scale = torch.cuda.amp.GradScaler()
        # optimizer.zero_grad()
        # print(f"{output_lane['loss']}")
        loss_lane = output_lane['loss'].sum()
        loss_cls_obj, loss_box_obj, loss_dfl_obj = criterion_obj(output_obj, targets)
        
        
        
        total_loss = loss_scaler*loss_lane + loss_cls_obj + loss_box_obj + loss_dfl_obj
        # print(f"{output_lane['loss_stats']}")
        # f.write(f"{output_lane['loss'].item():.6f}, {loss_obj.item():.6f}, {total_loss.item():.6f}\n")
        f.write(f"{total_loss.item():.6f}, {loss_cls_obj.item():.6f}, {loss_box_obj.item():.6f}, {loss_dfl_obj.item():.6f}, {output_lane['loss_stats']['cls_loss'].item():.6f}, {output_lane['loss_stats']['reg_xytl_loss'].item():.6f}, {output_lane['loss_stats']['seg_loss'].item():.6f}, {output_lane['loss_stats']['iou_loss'].item():.6f}\n")

        # p_bar.set_description(str(str(total_loss), str(loss_cls_obj), str(loss_box_obj), str(loss_dfl_obj), str(output_lane['loss_stats']['cls_loss']), str(output_lane['loss_stats']['reg_xytl_loss']), str(output_lane['loss_stats']['seg_loss']), str(output_lane['loss_stats']['iou_loss'])))
        p_bar.set_description(f"{total_loss.item():.6f}, {loss_cls_obj.item():.6f}, {loss_box_obj.item():.6f}, {loss_dfl_obj.item():.6f}, {output_lane['loss_stats']['cls_loss'].item():.6f}, {output_lane['loss_stats']['reg_xytl_loss'].item():.6f}, {output_lane['loss_stats']['seg_loss'].item():.6f}, {output_lane['loss_stats']['iou_loss'].item():.6f}")
        amp_scale.scale(total_loss).backward()
        # if i%2:
        #     amp_scale.scale(loss_lane).backward()
        # else:
        #     amp_scale.scale(loss_obj).backward()
        
        if x % accumulate == 0:
            amp_scale.unscale_(optimizer)  # unscale gradients
            util.clip_gradients(net)  # clip gradients
            amp_scale.step(optimizer)  # optimizer.step
            amp_scale.update()
            optimizer.zero_grad()
        # optimizer.step()
        # del loss_lane, loss_obj, total_loss, output_lane, output_obj, samples, targets

        
    
    print(f"Epoch {epoch} finished")
    net.eval()
    test_obj(args, params, net)
    # validate_lane(net, recorder)
    
    

f.close()
    
    
    
    
    
    

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
file_path = "loss_log.txt"  # Update this with the correct file path
df = pd.read_csv(file_path, sep=',', header=None)

df.columns = ["Loss Lane", "Loss Obj", "Loss Total"]

# Moving Average Smoothing
def smooth(y, box_pts=50):
    box = np.ones(box_pts) / box_pts
    return np.convolve(y, box, mode='same')

# Plot
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-darkgrid')

for column in df.columns:
    plt.plot(df.index, smooth(df[column], box_pts=100), label=column, linewidth=2)

# Titles and Labels
plt.xlabel("Batch", fontsize=14)
plt.ylabel("Loss", fontsize=14)
plt.title("Smoothed Loss Over Batches", fontsize=16, fontweight='bold')
plt.legend(fontsize=12)

# Enhanced Grid
plt.grid(True, linestyle="-", linewidth=1, color="gray", alpha=0.7)

# Show plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
file_path = "loss_log_randomized_weights.txt"  # Update this with the correct file path
df = pd.read_csv(file_path, sep=',', header=None)

df.columns = ["Loss Lane", "Loss Obj", "Loss Total"]

# Moving Average Smoothing
def smooth(y, box_pts=50):
    box = np.ones(box_pts) / box_pts
    return np.convolve(y, box, mode='same')

# Plot
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-darkgrid')

for column in df.columns:
    plt.plot(df.index, smooth(df[column], box_pts=100), label=column, linewidth=2)

# Titles and Labels
plt.xlabel("Batch", fontsize=14)
plt.ylabel("Loss", fontsize=14)
plt.title("Smoothed Loss Over Batches", fontsize=16, fontweight='bold')
plt.legend(fontsize=12)

# Enhanced Grid
plt.grid(True, linestyle="-", linewidth=1, color="gray", alpha=0.7)

# Show plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
file_path = "loss_log_randomized_weights_reverse_gradient.txt"  # Update this with the correct file path
df = pd.read_csv(file_path, sep=',', header=None)

df.columns = ["Loss Lane", "Loss Obj", "Loss Total"]

# Moving Average Smoothing
def smooth(y, box_pts=50):
    box = np.ones(box_pts) / box_pts
    return np.convolve(y, box, mode='same')

# Plot
plt.figure(figsize=(12, 6))
plt.style.use('seaborn-darkgrid')

for column in df.columns:
    plt.plot(df.index, smooth(df[column], box_pts=100), label=column, linewidth=2)

# Titles and Labels
plt.xlabel("Batch", fontsize=14)
plt.ylabel("Loss", fontsize=14)
plt.title("Smoothed Loss Over Batches", fontsize=16, fontweight='bold')
plt.legend(fontsize=12)

# Enhanced Grid
plt.grid(True, linestyle="-", linewidth=1, color="gray", alpha=0.7)

# Show plot
plt.show()

In [ ]:
args.epoch 

In [ ]:
net.eval()
validate_lane(net, recorder)

In [ ]:
from IPython.lib.backgroundjobs import BackgroundJobManager
import time

jobs = BackgroundJobManager()

import time
import cv2
import torch
from tqdm import tqdm
import pytorch_warmup as warmup
import numpy as np
import random
import os

from clrnet.models.registry import build_net
from clrnet.engine.registry import build_trainer, build_evaluator
from clrnet.engine.optimizer import build_optimizer
from clrnet.engine.scheduler import build_scheduler
from clrnet.datasets import build_dataloader
from clrnet.utils.recorder import build_recorder
from clrnet.utils.net_utils import save_model, load_network, resume_network
from clrnet.utils.recorder import build_recorder
from mmcv.parallel import MMDataParallel
from nets.nn import DarkNet
from utils import util

import argparse

import yaml
import os
import glob
from torch.utils import data


# yolo

from utils.dataset import Dataset
from clrnet.utils.config import Config

In [ ]:
folder_path = "/home/mmm9886/Desktop/Datasets/bdd100k/images/train/"
filenames = glob.glob(os.path.join(folder_path, '*'))

In [ ]:
%%script bash
nohup python -u -c '

import time
import cv2
import torch
from tqdm import tqdm
import pytorch_warmup as warmup
import numpy as np
import random
import os

from clrnet.models.registry import build_net
from clrnet.engine.registry import build_trainer, build_evaluator
from clrnet.engine.optimizer import build_optimizer
from clrnet.engine.scheduler import build_scheduler
from clrnet.datasets import build_dataloader
from clrnet.utils.recorder import build_recorder
from clrnet.utils.net_utils import save_model, load_network, resume_network
from clrnet.utils.recorder import build_recorder
from mmcv.parallel import MMDataParallel
from nets.nn import DarkNet
from utils import util

import argparse

import yaml
import os
import glob
from torch.utils import data


# yolo

from utils.dataset import Dataset
from clrnet.utils.config import Config


device = "cuda:0"

cfg = Config.fromfile("configs/clrnet/clr_resnet18_culane.py")

cfg.load_from = None
cfg.resume_from = None
cfg.finetune_from = None
cfg.view = None
cfg.seed = 0
cfg.gpus = 1

torch.manual_seed(cfg.seed)
np.random.seed(cfg.seed)
random.seed(cfg.seed)

train_loader_lane = build_dataloader(cfg.dataset.train,
                                    cfg,
                                    is_train=True)



parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])
args.input_size = 640
args.batch_size = 32
args.local_rank = 0
args.world_size = 1
args.epoch = 100


with open(os.path.join('utils', 'args.yaml'), errors='ignore') as f:
    params = yaml.safe_load(f)

folder_path = "/home/mmm9886/Desktop/Datasets/bdd100k/images/train/"
filenames = glob.glob(os.path.join(folder_path, '*'))

train_dataset_obj = Dataset(filenames, args.input_size, params, True)

train_loader_obj = data.DataLoader(train_dataset_obj, 18, False, num_workers=8,
                            pin_memory=True, collate_fn=Dataset.collate_fn)



state_dict = torch.load("/home/mmm9886/Desktop/Capstone_Implementation/YOLOv8-pt copy 3/weights/best.pt")
darknet_backbone = state_dict['model'].net
darknet_backbone = darknet_backbone.float()
darknet_backbone = darknet_backbone.to(device)
darknet_neck = state_dict['model'].fpn
darknet_neck = darknet_neck.float()
darknet_neck = darknet_neck.to(device)
darknet_head = state_dict['model'].head
darknet_head = darknet_head.float()
darknet_head = darknet_head.to(device)

net = build_net(cfg)

clrnet_head = net.heads

net = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)

accumulate = max(round(64 / (args.batch_size * args.world_size)), 1)
net = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)
net = net.to(device)

optimizer = build_optimizer(cfg, net)
scheduler = build_scheduler(cfg, optimizer)
criterion_obj = util.ComputeLoss(net, params)

net.train()
end = time.time()
max_iter = len(train_loader_lane)



zipped_dls = zip(train_loader_obj, train_loader_lane)

p_bar = enumerate(zipped_dls)
p_bar = tqdm(p_bar, total=len(train_loader_lane))

# for i, data in enumerate(train_loader_lane):
for i, (data_obj, data_lane) in p_bar:
    x = i + max_iter #* epoch
    date_time = time.time() - end
    samples = data_obj[0].to(device).float() / 255
    targets = data_obj[1].to(device)
    # data_obj = to_cuda(data_obj)
    data_lane = to_cuda(data_lane, device)
    output_obj = net(samples, 1)
    
    
    # experiment = net2(data_lane)
    output_lane = net(data_lane, 0)
    
    amp_scale = torch.cuda.amp.GradScaler()
    # optimizer.zero_grad()
    loss_lane = output_lane['loss'].sum()
    loss_obj = criterion_obj(output_obj, targets)
    total_loss = loss_lane+loss_obj
    p_bar.set_description(str(total_loss))
    amp_scale.scale(total_loss).backward()
    
    if x % accumulate == 0:
        amp_scale.unscale_(optimizer)  # unscale gradients
        util.clip_gradients(net)  # clip gradients
        amp_scale.step(optimizer)  # optimizer.step
        amp_scale.update()
        optimizer.zero_grad()
    # optimizer.step()
    del loss_lane, loss_obj, total_loss, output_lane, output_obj, samples, targets, data_lane
' > output.log 2>&1 &
echo $! > process_id.txt

In [ ]:
from IPython.lib.backgroundjobs import BackgroundJobManager
import time

jobs = BackgroundJobManager()

import time
import cv2
import torch
from tqdm import tqdm
import pytorch_warmup as warmup
import numpy as np
import random
import os

from clrnet.models.registry import build_net
from clrnet.engine.registry import build_trainer, build_evaluator
from clrnet.engine.optimizer import build_optimizer
from clrnet.engine.scheduler import build_scheduler
from clrnet.datasets import build_dataloader
from clrnet.utils.recorder import build_recorder
from clrnet.utils.net_utils import save_model, load_network, resume_network
from clrnet.utils.recorder import build_recorder
from mmcv.parallel import MMDataParallel
from nets.nn import DarkNet
from utils import util

import argparse

import yaml
import os
import glob
from torch.utils import data


# yolo

from utils.dataset import Dataset
from clrnet.utils.config import Config


def long_running_task():
    device = 'cuda:0'

    cfg = Config.fromfile("configs/clrnet/clr_resnet18_culane.py")
    
    cfg.load_from = None
    cfg.resume_from = None
    cfg.finetune_from = None
    cfg.view = None
    cfg.seed = 0
    cfg.gpus = 1
    
    torch.manual_seed(cfg.seed)
    np.random.seed(cfg.seed)
    random.seed(cfg.seed)
    
    train_loader_lane = build_dataloader(cfg.dataset.train,
                                        cfg,
                                        is_train=True)
    
    
    
    parser = argparse.ArgumentParser()
    args = parser.parse_args(args=[])
    args.input_size = 640
    args.batch_size = 32
    args.local_rank = 0
    args.world_size = 1
    args.epoch = 100
    
    
    with open(os.path.join('utils', 'args.yaml'), errors='ignore') as f:
        params = yaml.safe_load(f)

    folder_path = "/home/mmm9886/Desktop/Datasets/bdd100k/images/train/"
    filenames = glob.glob(os.path.join(folder_path, '*'))

    train_dataset_obj = Dataset(filenames, args.input_size, params, True)
    
    train_loader_obj = data.DataLoader(train_dataset_obj, 18, False, num_workers=8,
                             pin_memory=True, collate_fn=Dataset.collate_fn)
    
    
    
    state_dict = torch.load("/home/mmm9886/Desktop/Capstone_Implementation/YOLOv8-pt copy 3/weights/best.pt")
    darknet_backbone = state_dict['model'].net
    darknet_backbone = darknet_backbone.float()
    darknet_backbone = darknet_backbone.to(device)
    darknet_neck = state_dict['model'].fpn
    darknet_neck = darknet_neck.float()
    darknet_neck = darknet_neck.to(device)
    darknet_head = state_dict['model'].head
    darknet_head = darknet_head.float()
    darknet_head = darknet_head.to(device)
    
    net = build_net(cfg)
    
    clrnet_head = net.heads
    
    net = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)
    
    
    accumulate = max(round(64 / (args.batch_size * args.world_size)), 1)
    net = YOLOL(darknet_backbone, darknet_neck, darknet_head, clrnet_head)
    net = net.to(device)

    optimizer = build_optimizer(cfg, net)
    scheduler = build_scheduler(cfg, optimizer)
    criterion_obj = util.ComputeLoss(net, params)

    net.train()
    end = time.time()
    max_iter = len(train_loader_lane)



    zipped_dls = zip(train_loader_obj, train_loader_lane)

    p_bar = enumerate(zipped_dls)
    p_bar = tqdm(p_bar, total=len(train_loader_lane))

    # for i, data in enumerate(train_loader_lane):
    for i, (data_obj, data_lane) in p_bar:
        x = i + max_iter #* epoch
        date_time = time.time() - end
        samples = data_obj[0].to(device).float() / 255
        targets = data_obj[1].to(device)
        # data_obj = to_cuda(data_obj)
        data_lane = to_cuda(data_lane, device)
        output_obj = net(samples, 1)
        
        
        # experiment = net2(data_lane)
        output_lane = net(data_lane, 0)
        
        amp_scale = torch.cuda.amp.GradScaler()
        # optimizer.zero_grad()
        loss_lane = output_lane['loss'].sum()
        loss_obj = criterion_obj(output_obj, targets)
        total_loss = loss_lane+loss_obj
        p_bar.set_description(str(total_loss))
        amp_scale.scale(total_loss).backward()
        
        if x % accumulate == 0:
            amp_scale.unscale_(optimizer)  # unscale gradients
            util.clip_gradients(net)  # clip gradients
            amp_scale.step(optimizer)  # optimizer.step
            amp_scale.update()
            optimizer.zero_grad()
        # optimizer.step()
        del loss_lane, loss_obj, total_loss, output_lane, output_obj, samples, targets, data_lane
        
jobs.new(long_running_task)

print("Task started in the background. You can disconnect safely.")



In [ ]:
jobs.flush()

In [ ]:
jobs.flush()

In [ ]:
net

In [ ]:
net.eval()
test_obj(args, params, net)

In [ ]:
net.eval()
validate_lane(net, recorder)

In [ ]:
torch.save(net.state_dict(), 'net_model.pth')

In [ ]:
net

In [ ]:
args.epoch

In [ ]:
# p_bar = enumerate(train_loader_obj)
# for i, data in p_bar:
#     print(data[1])
#     print("target")
#     print(target)
#     break

In [ ]:
# zipped_dls = zip(train_loader_obj, train_loader_lane)
# # for i, data in enumerate(train_loader_lane):
# for i, (data_obj, data_lane) in enumerate(zipped_dls):
#     zip = data_obj
#     break

In [ ]:
# for i, (samples, target, extra) in p_bar:
#     break

In [ ]:
# print(zip[0].shape)
# print(zip[1].shape)
# print(len(zip[2]))

In [ ]:
samples.shape

In [ ]:
target.shape

In [ ]:
extra

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
model.train(
            data="/home/mmm9886/Desktop/Datasets/bdd100k/bdd100k.yaml", 
            project="runs/yolo_bdd100k_benchmark",
            epochs=50, 
            batch=16, 
            imgsz=640)

In [ ]:
from ultralytics import YOLO
model = YOLO("/home/mmm9886/Desktop/Capstone_Implementation/clrnet/runs/yolo_bdd100k_benchmark/train3/weights/best.pt")

In [ ]:
model.val(conf = 0.25, iou = 0.45)